# Complete Workflow Verification - Self-Contained

This notebook demonstrates and verifies the complete workflow as documented in `experiment_v2_workflow_implementation.md`.

```
┌─────────────────────────────────────────────┐
│      CUSTOM WORKFLOW - USER FLOW            │
└─────────────────────────────────────────────┘

                START
                  ↓
    ┌─────────────────────────────┐
    │  1. Create Experiment       │
    └─────────────────────────────┘
                  ↓
    ┌─────────────────────────────┐
    │  2. Get Placeholder Traces  │
    │     (with inputs)           │
    └─────────────────────────────┘
                  ↓
    ┌─────────────────────────────┐
    │  3. Process with Your Code  │
    │     (external)              │
    └─────────────────────────────┘
                  ↓
    ┌─────────────────────────────┐
    │  4. Submit Results          │
    └─────────────────────────────┘
                  ↓
    ┌─────────────────────────────┐
    │  5. View Final Results      │
    │     (with scores)           │
    └─────────────────────────────┘
                  ↓
                 END
```


## Configuration

Set your API key and base URL here.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
loaded = load_dotenv(override=True)

# API Configuration
BASE_URL = os.getenv("KEYWORDSAI_BASE_URL")
API_KEY = os.getenv("KEYWORDSAI_API_KEY")

if not API_KEY:
    raise ValueError(
        "❌ KEYWORDSAI_API_KEY environment variable not set!\n"
        "Make sure you have a .env file with: KEYWORDSAI_API_KEY=your-api-key-here\n"
        "Or set it with: export KEYWORDSAI_API_KEY='your-api-key-here'"
    )

print(f"Loaded .env from {os.getcwd()}/.env: {loaded}")
print(f"✅ API Key loaded: {API_KEY[:8]}{'*' * 20}")


## Imports and All Functions

All necessary imports and function definitions in one place.


In [ ]:
# Standard library imports
import json
import time
import requests
from typing import Dict, Any, List, Optional
from datetime import datetime, timedelta

# ============================================================================
# CONSISTENT NAMING FOR ALL COMPONENTS - MAKES CLEANUP EASY!
# ============================================================================
WORKFLOW_NAME = "Custom workflow test"

# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def print_step(step_number: int, title: str):
    """Print a formatted step header."""
    print("\n" + "=" * 70)
    print(f"STEP {step_number}: {title}")
    print("=" * 70)

def print_success(message: str):
    """Print a success message."""
    print(f"✅ {message}")

def print_warning(message: str):
    """Print a warning message."""
    print(f"⚠️  {message}")

def print_error(message: str):
    """Print an error message."""
    print(f"❌ {message}")

def print_info(message: str):
    """Print an info message."""
    print(f"ℹ️  {message}")

def wait_for_processing(seconds: int = 15):
    """Wait for async processing to complete."""
    print(f"\nWaiting {seconds} seconds for processing...")
    time.sleep(seconds)
    print("✓ Wait complete")

# ============================================================================
# API FUNCTIONS
# ============================================================================

def create_log(
    model: str,
    input_messages: List[Dict[str, str]],
    output_message: Dict[str, str],
    custom_identifier: Optional[str] = None,
    span_name: Optional[str] = None,
    **kwargs
) -> Dict[str, Any]:
    """Create a new log entry in Keywords AI."""
    url = f"{BASE_URL}/request-logs/create"
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}"
    }
    
    payload = {
        "model": model,
        "input": input_messages,
        "output": output_message
    }
    
    if custom_identifier:
        payload["custom_identifier"] = custom_identifier
    if span_name:
        payload["span_name"] = span_name
    
    payload.update(kwargs)
    
    print("Creating log entry...")
    print(f"  URL: {url}")
    print(f"  Model: {model}")
    print(f"  Input messages: {len(input_messages)}")
    if custom_identifier:
        print(f"  Custom identifier: {custom_identifier}")
    if span_name:
        print(f"  Span name: {span_name}")
    print(f"  Request Body: {json.dumps(payload, indent=2)}")
    
    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    
    data = response.json()
    print(f"\n✓ Log created successfully")
    if 'unique_id' in data:
        print(f"  Log ID (unique_id): {data.get('unique_id')}")
    if 'id' in data:
        print(f"  Log ID: {data.get('id')}")
    if 'trace_id' in data:
        print(f"  Trace ID: {data.get('trace_id')}")
    
    return data

def create_dataset(
    name: str, 
    description: str = "", 
    dataset_type: str = "sampling",
    sampling: int = 50,
    start_time: Optional[str] = None,
    end_time: Optional[str] = None,
    initial_log_filters: Optional[Dict[str, Any]] = None
) -> Dict[str, Any]:
    """Create a new dataset from logs with sampling and filtering."""
    url = f"{BASE_URL}/datasets"
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}"
    }
    
    payload = {
        "name": name,
        "description": description,
        "type": dataset_type
    }
    
    if sampling and not initial_log_filters:
        payload["sampling"] = sampling
    
    if start_time:
        payload["start_time"] = start_time
    if end_time:
        payload["end_time"] = end_time
        
    if initial_log_filters:
        payload["initial_log_filters"] = initial_log_filters
    
    print("Creating dataset...")
    print(f"  URL: {url}")
    print(f"  Name: {name}")
    print(f"  Type: {dataset_type}")
    print(f"  Request Body: {json.dumps(payload, indent=2)}")
    
    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    
    data = response.json()
    print(f"\n✓ Dataset created successfully")
    print(f"  Dataset ID: {data.get('id')}")
    print(f"  Name: {data.get('name')}")
    print(f"  Status: {data.get('status', 'N/A')}")
    
    return data

def list_dataset_logs(dataset_id: str, page: int = 1, page_size: int = 100) -> Dict[str, Any]:
    """List logs from a specific dataset."""
    url = f"{BASE_URL}/datasets/{dataset_id}/logs/list"
    
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    
    params = {
        "page": page,
        "page_size": page_size
    }
    
    print(f"\nListing logs for dataset {dataset_id}...")
    print(f"  URL: {url}")
    print(f"  Method: GET")
    print(f"  Params: {params}")
    
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    
    data = response.json()
    results = data.get('results', [])
    total_count = data.get('count', 0)
    
    print(f"✓ Retrieved {len(results)} logs (page {page})")
    print(f"  Total logs in dataset: {total_count}")
    
    if results:
        print(f"\n  📋 First log structure:")
        first_log = results[0]
        print(f"    Keys: {list(first_log.keys())}")
        if 'id' in first_log:
            print(f"    Log ID: {first_log.get('id')}")
        if 'input' in first_log:
            input_preview = str(first_log.get('input'))[:100]
            print(f"    Input preview: {input_preview}...")
        if 'output' in first_log:
            output_preview = str(first_log.get('output'))[:100]
            print(f"    Output preview: {output_preview}...")
    
    return data

def create_evaluator(
    name: str,
    evaluator_slug: str,
    evaluator_type: str,
    score_value_type: str,
    description: str = "",
    configurations: Optional[Dict[str, Any]] = None
) -> Dict[str, Any]:
    """Create a custom evaluator in Keywords AI."""
    url = f"{BASE_URL}/evaluators"
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}"
    }
    
    payload = {
        "name": name,
        "evaluator_slug": evaluator_slug,
        "type": evaluator_type,
        "score_value_type": score_value_type,
        "description": description
    }
    
    if configurations:
        payload["configurations"] = configurations
    
    print("Creating evaluator...")
    print(f"  URL: {url}")
    print(f"  Name: {name}")
    print(f"  Slug: {evaluator_slug}")
    print(f"  Type: {evaluator_type}")
    print(f"  Score Type: {score_value_type}")
    print(f"  Request Body: {json.dumps(payload, indent=2)}")
    
    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    
    data = response.json()
    print(f"\n✓ Evaluator created successfully")
    if 'id' in data:
        print(f"  Evaluator ID: {data.get('id')}")
    if 'evaluator_slug' in data:
        print(f"  Evaluator Slug: {data.get('evaluator_slug')}")
    
    return data

def create_experiment(name: str, description: str, dataset_id: str, 
                     workflow: List[Dict], evaluator_slugs: List[str]) -> Dict[str, Any]:
    """Create a new custom workflow experiment."""
    url = f"{BASE_URL}/v2/experiments/"
    
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "name": name,
        "description": description,
        "dataset_id": dataset_id,
        "workflow": workflow,
        "evaluator_slugs": evaluator_slugs
    }
    
    print("Creating custom workflow experiment...")
    print(f"  URL: {url}")
    print(f"  Name: {name}")
    print(f"  Dataset: {dataset_id}")
    print(f"  Evaluators: {', '.join(evaluator_slugs)}")
    print(f"  Request Body: {json.dumps(payload, indent=2)}")
    
    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    
    data = response.json()
    print(f"\n✓ Experiment created with ID: {data.get('id')}")
    print(f"  Status: {data.get('status')}")
    print("  Placeholder traces are being created asynchronously...")
    
    return data

def list_experiment_logs(exp_id: str, filters: Optional[Dict[str, Any]] = None, 
                        page: int = 1, page_size: int = 100) -> Dict[str, Any]:
    """List traces for an experiment with optional filtering."""
    url = f"{BASE_URL}/v2/experiments/{exp_id}/logs/list/"
    
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    
    params = {
        "page": page,
        "page_size": page_size
    }
    
    print(f"\nListing experiment logs for {exp_id}...")
    print(f"  URL: {url}")
    print(f"  Params: {params}")
    if filters:
        print(f"  Filters: {json.dumps(filters, indent=2)}")
    
    try:
        if filters:
            request_body = {"filters": filters}
            print(f"  Method: POST")
            print(f"  Request Body: {json.dumps(request_body, indent=2)}")
            response = requests.post(url, headers=headers, json=request_body, params=params)
        else:
            print(f"  Method: GET")
            response = requests.get(url, headers=headers, params=params)
        
        response.raise_for_status()
        
    except requests.exceptions.HTTPError as e:
        print(f"❌ Error listing logs: {e}")
        print(f"   Status code: {e.response.status_code}")
        
        try:
            error_json = e.response.json()
            print(f"   Response: {error_json}")
        except:
            print(f"   Response (text): {e.response.text[:500]}")
        
        if filters and e.response.status_code == 500:
            print("\n⚠ Retrying without filters...")
            try:
                response = requests.get(url, headers=headers, params=params)
                response.raise_for_status()
                print("✓ GET without filters succeeded")
            except Exception as retry_error:
                print(f"❌ Retry also failed: {retry_error}")
                raise e
        else:
            raise
    
    data = response.json()
    results = data.get('results', [])
    print(f"✓ Found {len(results)} logs (page {page})")
    print(f"  Total count: {data.get('count', 0)}")
    
    if results:
        status_counts = {}
        for log in results:
            status = log.get('status', 'unknown')
            status_counts[status] = status_counts.get(status, 0) + 1
        print(f"  Status breakdown: {status_counts}")
    
    return data

def get_trace_details(exp_id: str, trace_id: str, include_full_span_tree: bool = True) -> Dict[str, Any]:
    """Get detailed information about a specific trace."""
    url = f"{BASE_URL}/v2/experiments/{exp_id}/logs/{trace_id}/"
    
    headers = {
        "Authorization": f"Bearer {API_KEY}"
    }
    
    params = {}
    if include_full_span_tree:
        params["detail"] = 1
    
    print(f"  Getting trace details...")
    print(f"    URL: {url}")
    print(f"    Method: GET")
    if params:
        print(f"    Params: {params}")
    
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    
    return response.json()

def submit_workflow_results(exp_id: str, trace_id: str, 
                           input_data: Any, output_data: Any,
                           name: Optional[str] = None, 
                           metadata: Optional[Dict] = None) -> Dict[str, Any]:
    """Update a placeholder trace with your custom workflow results."""
    url = f"{BASE_URL}/v2/experiments/{exp_id}/logs/{trace_id}/"
    
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "input": input_data,
        "output": output_data
    }
    
    if name:
        payload["name"] = name
    if metadata:
        payload["metadata"] = metadata
    
    print(f"    Submitting results to: {url}")
    print(f"    Method: PATCH")
    print(f"    Payload keys: {list(payload.keys())}")
    print(f"    Input type: {type(input_data).__name__}")
    print(f"    Output type: {type(output_data).__name__}")
    
    response = requests.patch(url, headers=headers, json=payload)
    response.raise_for_status()
    
    updated_trace = response.json()
    
    print(f"    Response status: {response.status_code}")
    
    response_status = updated_trace.get('status')
    if response_status:
        print(f"    Trace status: {response_status}")
    
    return updated_trace

def get_experiment_summary(exp_id: str, filters: Optional[List[Dict]] = None) -> Dict[str, Any]:
    """Get aggregated summary statistics for experiment traces."""
    url = f"{BASE_URL}/v2/experiments/{exp_id}/logs/summary/"
    
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    
    print(f"\nGetting experiment summary for {exp_id}...")
    print(f"  URL: {url}")
    
    if filters:
        request_body = {"filters": filters}
        print(f"  Method: POST")
        print(f"  Request Body: {json.dumps(request_body, indent=2)}")
        response = requests.post(url, headers=headers, json=request_body)
    else:
        print(f"  Method: GET")
        response = requests.get(url, headers=headers)
    
    response.raise_for_status()
    
    data = response.json()
    print(f"✓ Summary retrieved:")
    print(f"  Total traces: {data.get('total_count', 0)}")
    print(f"  Total cost: ${data.get('total_cost', 0):.4f}")
    print(f"  Total tokens: {data.get('total_tokens', 0)}")
    print(f"  Avg latency: {data.get('avg_latency', 0):.2f}s")
    
    return data

def process_with_custom_logic(input_data: Any) -> Dict[str, Any]:
    """Your custom workflow processing logic."""
    try:
        if isinstance(input_data, str):
            try:
                parsed_input = json.loads(input_data)
            except:
                parsed_input = input_data
        else:
            parsed_input = input_data
        
        result = {
            "status": "processed",
            "message": f"Successfully processed input with {len(str(parsed_input))} characters",
            "input_preview": str(parsed_input)[:100],
            "custom_field": "your_custom_value"
        }
        
        return {
            "output": result,
            "metadata": {
                "processing_timestamp": time.time(),
                "processor": "custom_workflow_v1"
            }
        }
        
    except Exception as e:
        return {
            "output": {
                "status": "error",
                "error_message": str(e)
            },
            "metadata": {
                "processing_timestamp": time.time(),
                "processor": "custom_workflow_v1",
                "error": True
            }
        }


## Initialize Variables

These variables will track resources created throughout the workflow.


In [ ]:
# Track resources created
log_ids = []
dataset_id = None
evaluator_slug = None
experiment_id = None
trace_ids = []
dataset_log_count = 0
trace_count = 0
processed_count = 0
traces_with_evaluators = 0
final_status_breakdown = {}


## Step 1: Create Sample Logs

Create 3 sample logs that will be added to the dataset.


In [ ]:
print_step(1, "Create Sample Logs")

log_examples = [
    {
        "model": "gpt-4",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "What is machine learning?"}
        ],
        "response": "Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience."
    },
    {
        "model": "gpt-4",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Explain neural networks briefly."}
        ],
        "response": "Neural networks are computing systems inspired by biological neural networks in animal brains."
    },
    {
        "model": "gpt-4",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "What is deep learning?"}
        ],
        "response": "Deep learning is a type of machine learning based on artificial neural networks with multiple layers."
    }
]

for i, example in enumerate(log_examples, 1):
    print(f"\nCreating log {i}/{len(log_examples)}...")
    log_result = create_log(
        model=example["model"],
        input_messages=example["messages"],
        output_message={"role": "assistant", "content": example["response"]},
        custom_identifier=f"{WORKFLOW_NAME}_log_{i}",
        span_name=WORKFLOW_NAME
    )
    log_id = log_result.get('unique_id') or log_result.get('id')
    if log_id:
        log_ids.append(log_id)
        print_success(f"Log created with ID: {log_id[:16]}...")

print_success(f"Created {len(log_ids)} logs total")
print_info("Waiting for logs to persist in database...")
wait_for_processing(15)


## Step 2: Create Dataset from Logs

Create a dataset using the logs we just created.


In [ ]:
print_step(2, "Create Dataset from Logs")

end_time = datetime.utcnow()
start_time = end_time - timedelta(days=2)

initial_filters = {
    "id": {
        "value": log_ids,
        "operator": "in"
    }
}

dataset_result = create_dataset(
    name=WORKFLOW_NAME,
    description="Dataset created from complete self-contained notebook",
    dataset_type="sampling",
    start_time=start_time.isoformat() + "Z",
    end_time=end_time.isoformat() + "Z",
    initial_log_filters=initial_filters
)

dataset_id = dataset_result.get('id')
print_success(f"Dataset created with ID: {dataset_id}")
print_info("Waiting for dataset to populate...")
wait_for_processing(15)


## Step 3: Verify Dataset Contains the Logs 

This is a critical verification step to ensure the dataset actually contains the logs we created.


In [ ]:
print_step(3, "Verify Dataset Contains the Logs")

dataset_logs = list_dataset_logs(dataset_id, page=1, page_size=100)
dataset_log_count = len(dataset_logs.get('results', []))
total_dataset_logs = dataset_logs.get('count', 0)

print(f"Expected logs: {len(log_ids)}")
print(f"Dataset contains: {total_dataset_logs} logs")
print(f"Retrieved for verification: {dataset_log_count} logs")

if dataset_log_count == 0:
    print_error("Dataset is empty! Workflow cannot proceed.")
    raise Exception("Dataset is empty")
elif dataset_log_count < len(log_ids):
    print_warning(f"Expected {len(log_ids)} but found {dataset_log_count}")
    print_info("Some logs may not have been added yet, but proceeding...")
else:
    print_success(f"Dataset successfully populated with all {dataset_log_count} logs!")

if dataset_log_count > 0:
    first_log = dataset_logs['results'][0]
    print(f"\n📋 Sample log from dataset:")
    print(f"  ID: {first_log.get('id', 'N/A')[:24]}...")
    if 'input' in first_log:
        print(f"  Has input: Yes ({len(str(first_log['input']))} chars)")
    if 'output' in first_log:
        print(f"  Has output: Yes ({len(str(first_log['output']))} chars)")


## Step 4: Create Custom Evaluator

Create an LLM-based evaluator that rates response quality on a 1-5 scale based on accuracy, relevance, and completeness.


In [ ]:
print_step(4, "Create Custom Evaluator")

evaluator_slug = f"custom_workflow_test_eval_{int(time.time())}"

evaluator_result = create_evaluator(
    name=WORKFLOW_NAME,
    evaluator_slug=evaluator_slug,
    evaluator_type="llm",
    score_value_type="numerical",
    description="Evaluates response quality on a 1-5 scale",
    configurations={
        "evaluator_definition": "Rate the response quality based on accuracy, relevance, and completeness.\n<llm_input>{{llm_input}}</llm_input>\n<llm_output>{{llm_output}}</llm_output>",
        "scoring_rubric": "1=Poor, 2=Fair, 3=Good, 4=Very Good, 5=Excellent",
        "llm_engine": "gpt-4o-mini",
        "model_options": {
            "temperature": 0.1,
            "max_tokens": 200
        },
        "min_score": 1.0,
        "max_score": 5.0,
        "passing_score": 3.0
    }
)

print_success(f"Evaluator created with slug: {evaluator_slug}")


## Step 5: Create Experiment with Dataset and Evaluator

Create a custom workflow experiment that will create placeholder traces for each dataset entry.


In [ ]:
print_step(5, "Create Experiment with Dataset and Evaluator")

experiment_data = create_experiment(
    name=WORKFLOW_NAME,
    description="Testing complete self-contained workflow",
    dataset_id=dataset_id,
    workflow=[{
        "type": "custom",
        "config": {
            "name": WORKFLOW_NAME,
            "description": "Custom workflow from complete notebook"
        }
    }],
    evaluator_slugs=[evaluator_slug]
)

experiment_id = experiment_data.get('id')
print_success(f"Experiment created with ID: {experiment_id}")
print_info("Placeholder traces being created asynchronously...")


## Step 6: Wait for Traces and List Placeholder Traces

Wait for async trace creation and retrieve the placeholder traces with retry logic.


In [ ]:
print_step(6, "Wait for Traces and List Placeholder Traces")

print_info("Waiting for placeholder traces (with retry)...")

trace_count = 0
max_retries = 6
retry_wait = 10

for attempt in range(1, max_retries + 1):
    print(f"  Attempt {attempt}/{max_retries}...")
    wait_for_processing(retry_wait)
    
    traces_response = list_experiment_logs(experiment_id, filters=None, page_size=10)
    trace_count = len(traces_response.get('results', []))
    
    if trace_count > 0:
        print_success(f"Found {trace_count} traces on attempt {attempt}")
        break
    else:
        if attempt < max_retries:
            print_info(f"No traces yet, waiting {retry_wait} more seconds...")

traces = list_experiment_logs(experiment_id, filters=None, page_size=10)
trace_count = len(traces.get('results', []))

print(f"\nFound {trace_count} traces")

if trace_count == 0:
    print_error("No traces found after multiple retries!")
    raise Exception("No traces found")

print_success(f"Successfully retrieved {trace_count} traces")

status_breakdown = {}
for trace in traces.get('results', []):
    status = trace.get('status', 'unknown')
    status_breakdown[status] = status_breakdown.get(status, 0) + 1

print(f"\nStatus breakdown:")
for status, count in status_breakdown.items():
    print(f"  {status}: {count}")


## Step 7: Process Traces with Custom Workflow

Process each trace with our custom logic and submit the results.


In [ ]:
print_step(7, "Process Traces with Custom Workflow")

processed_count = 0
max_to_process = min(trace_count, 3)
print(max_to_process)
for i, trace in enumerate(traces['results'][:max_to_process], 1):
    trace_id = trace.get('id')
    trace_ids.append(trace_id)
    
    print(f"\nProcessing trace {i}/{max_to_process} ({trace_id})...")
    
    input_data = trace.get('input')
    
    result = process_with_custom_logic(input_data)
    
    updated_trace = submit_workflow_results(
        exp_id=experiment_id,
        trace_id=trace_id,
        input_data=input_data,
        output_data=result['output'],
        name=WORKFLOW_NAME,
        metadata=result.get('metadata')
    )
    
    processed_count += 1
    print_success(f"Submitted results for trace {trace_id[:16]}...")

print_success(f"Processed and submitted {processed_count} traces")
print_info("Waiting for evaluators to execute...")
wait_for_processing(15)


## Step 8: Verify Experiment Results 

Get comprehensive results including status breakdown and evaluator execution.


In [ ]:
print_step(8, "Verify Experiment Results")

try:
    final_summary = get_experiment_summary(experiment_id)
    print(f"\n📊 Experiment Summary:")
    print(f"  Total traces: {final_summary.get('total_count', 0)}")
    print(f"  Total cost: ${final_summary.get('total_cost', 0):.4f}")
    print(f"  Total tokens: {final_summary.get('total_tokens', 0)}")
    print(f"  Avg latency: {final_summary.get('avg_latency', 0):.2f}s")
except Exception as e:
    print_warning(f"Could not get summary: {e}")
    final_summary = {}

all_traces = list_experiment_logs(experiment_id, filters=None)
final_status_breakdown = {}
for trace in all_traces.get('results', []):
    status = trace.get('status', 'unknown')
    final_status_breakdown[status] = final_status_breakdown.get(status, 0) + 1

print(f"\n📈 Final Status Breakdown:")
for status, count in final_status_breakdown.items():
    total = all_traces.get('count', 1)
    percentage = (count / total * 100) if total > 0 else 0
    print(f"  {status}: {count} ({percentage:.1f}%)")

print(f"\n🔍 Checking Evaluator Execution:")
traces_with_evaluators = 0

for trace_id in trace_ids[:3]:
    trace_detail = get_trace_details(experiment_id, trace_id, include_full_span_tree=True)
    span_tree = trace_detail.get('span_tree', [])
    
    evaluator_spans = [
        span for span in span_tree
        if span.get('span_type') == 'SCORE' or 'evaluator' in span.get('span_name', '').lower()
    ]
    
    if evaluator_spans:
        traces_with_evaluators += 1
        print(f"  ✓ Trace {trace_id[:16]}... has {len(evaluator_spans)} evaluator span(s)")
        for eval_span in evaluator_spans:
            span_name = eval_span.get('span_name', 'unknown')
            score = eval_span.get('score', 'N/A')
            print(f"    - {span_name}: score = {score}")
    else:
        print(f"  ⚠️  Trace {trace_id[:16]}... has no evaluator results yet")

if traces_with_evaluators > 0:
    print_success(f"Evaluators executed on {traces_with_evaluators}/{len(trace_ids[:3])} sampled traces")
else:
    print_warning("No evaluator results found (may still be processing)")


## Step 9: Debug - View Full Span Tree Structure

Let's inspect the actual span tree structure to see where the evaluator results are.


In [ ]:
print_step(9, "Debug - View Full Span Tree Structure")

# Wait a bit more to ensure evaluators have completed
print_info("Waiting 20 seconds to ensure evaluators have fully completed...")
wait_for_processing(20)

# Get first trace with full details
if trace_ids:
    first_trace_id = trace_ids[0]
    print(f"\n🔍 Inspecting trace: {first_trace_id}")
    
    trace_detail = get_trace_details(experiment_id, first_trace_id, include_full_span_tree=True)
    
    print(f"\n📋 Trace-level keys: {list(trace_detail.keys())}")
    
    span_tree = trace_detail.get('span_tree', [])
    print(f"\n🌲 Span Tree: {len(span_tree)} total spans")
    
    if not span_tree:
        print_error("Span tree is empty!")
    else:
        for i, span in enumerate(span_tree, 1):
            print(f"\n  [{i}] Span Details:")
            print(f"      span_name: {span.get('span_name', 'N/A')}")
            print(f"      span_type: {span.get('span_type', 'N/A')}")
            print(f"      log_type: {span.get('log_type', 'N/A')}")
            print(f"      status: {span.get('status', 'N/A')}")
            
            # Show all keys to see what's available
            print(f"      Available keys: {list(span.keys())}")
            
            # Check for score-related fields
            if 'score' in span:
                print(f"      ⭐ score: {span['score']}")
            if 'evaluation_result' in span:
                print(f"      evaluation_result: {span['evaluation_result']}")
            if 'evaluator_slug' in span:
                print(f"      evaluator_slug: {span['evaluator_slug']}")
            
            # Show output if it's small enough
            output = span.get('output', '')
            if output:
                output_str = str(output)
                if len(output_str) > 200:
                    print(f"      output: {output_str[:200]}...")
                else:
                    print(f"      output: {output_str}")
            
            # Check if this span has children
            if 'children' in span:
                children = span.get('children', [])
                print(f"      children: {len(children)} child span(s)")
                if children:
                    for j, child in enumerate(children, 1):
                        print(f"        [{j}] {child.get('span_name', 'N/A')} (type: {child.get('span_type', 'N/A')})")
else:
    print_warning("No trace IDs available to inspect")
